In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI as genai
from langchain_community.document_loaders import WebBaseLoader
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings as GeminiEmbeddings

In [9]:
load_dotenv()
_DEFAULT_EMBEDDINGS_MODEL= "models/embedding-001"
# Initializes a large language model (LLM) for text generation
llm = genai(model="models/gemini-2.0-flash-exp",
            temperature=0.6,
             top_p=0.85,
            )

In [10]:
question = "what are the most recent udpates about hurricane Helene as of September 2024?"
response = llm.invoke(question)
print(response.content)

As of today, September 26, 2024, there is **no active hurricane named Helene**.

Here's why and how to find the most accurate, real-time information:

*   **Hurricane Season and Naming:** Hurricane names are recycled on a six-year cycle, unless a storm is so devastating that its name is retired. Helene was used in the past, but there is no active storm with that name currently.
*   **Where to Get Real-Time Information:** The most reliable sources for up-to-the-minute hurricane information are:

    *   **National Hurricane Center (NHC):** This is the official source for hurricane forecasts and warnings in the Atlantic and Eastern Pacific. Their website is hurricanes.gov.
    *   **National Weather Service (NWS):** The NWS provides local weather forecasts and warnings.
    *   **Reputable Weather News Outlets:** The Weather Channel, AccuWeather, and major news networks (CNN, BBC, etc.) provide coverage based on information from the NHC and NWS.

**In summary, there is no current hurrica

In [11]:
url = "https://www.cnn.com/weather/live-news/hurricane-helene-florida-north-carolina-georgia-09-30-24/index.html"

loader = WebBaseLoader(url)

docs = loader.load()

In [12]:
print(docs[0].page_content)

 

































September 30 news on Hurricane Helene | CNN











































































CNN values your feedback




                                                        1. How relevant is this ad to you?
                                                






























                                                2. Did you encounter any technical issues?
                                        











                                                                        Video player was slow to load content
                                                                        



                                                                        Video content never loaded
                                                                        



                                                                        Ad froze or did not finish loading
                                    

In [13]:
embeddings = GeminiEmbeddings(model=_DEFAULT_EMBEDDINGS_MODEL)
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

** Context **
{context}

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [15]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [16]:
response = retrieval_chain.invoke({"input": question})

In [17]:
print(f"Prompt/> {question}\n\n", f"Answer: {response['answer']}")

Prompt/> what are the most recent udpates about hurricane Helene as of September 2024?

 Answer: Here are the most recent updates about Hurricane Helene as of September 2024, based on the provided context:

*   **October 1, 2024:** At least 130 people have died across six states, and officials fear the death toll could rise. Many more remain missing. Hundreds of roads remain closed, especially in the Carolinas, hampering the delivery of badly-needed supplies. More than 2 million customers remain without power.
*   Urban search and rescue teams will finish all their operations in the state of Florida by the end of the day Monday.
*   Florida Gov. Ron DeSantis said multiple state agencies are also focusing on deploying recovery assistance to hard-hit areas in North Carolina and Tennessee, which have been devastated by flooding.
*   Buncombe County, North Carolina – which includes the city of Asheville – now has nearly three dozen confirmed deaths from Hurricane Helene, bringing the total